# Lesson 6 : EPICS area detector

Start the instrument package as our routine initialization.

In [ ]:
from instrument.collection import *

The EPICS area detector support in ophyd has many features, some of them specific to the detector make and model.  Let's start with just a few features using the ADSimDetector.  We'll need to import support from ophyd by parts as they are needed.

As we add features from the area detector plugins or other capabilities, the configuration complexity will increase.  So, it is good to start with a simple case where we can control the camera and generate images.

In [ ]:
from ophyd import Component
from ophyd.areadetector import ImagePlugin
from ophyd.areadetector import SimDetector
from ophyd.areadetector import SingleTrigger

In [ ]:
# for jupyterlab ...
# https://stackoverflow.com/a/51932652
%matplotlib inline

In [ ]:
_ad_prefix = "adsky:"

In [ ]:
class MySingleTriggerSimDetector(SingleTrigger, SimDetector): 
       
    image = Component(ImagePlugin, suffix="image1:")

In [ ]:
adsimdet = MySingleTriggerSimDetector(_ad_prefix, name='adsimdet')
adsimdet.wait_for_connection()

In [ ]:
adsimdet.stage_sigs["cam.num_images"] = 1
adsimdet.stage_sigs["cam.acquire_time"] = 0.1
adsimdet.stage_sigs["cam.acquire_period"] = 0.25

adsimdet.stage_sigs

In [ ]:
adsimdet.stage_sigs

In [ ]:
RE(bp.count([adsimdet]))

## Display image from EPICS PV data

Get the detector image and display it.  In the initial instrument setup, MatPlotLib was initialized for our display.  We need a copy of the plotting object that was created, `plt` to show the detector image.

In [ ]:
# get the MatPlotLib tool
from instrument.mpl import plt

In [ ]:
# NOTE:
#   These next cells show how one might get the image data directly from EPICS
#   This is not recommended practice with Bluesky so the code has been commented out.

# # The image comes from EPICS AD as a 1-D array, row by row.
# # We need to make it into a 2-D array, so first we need to
# # get the number of rows and columns from the image plugin.

# array_size = adsimdet.image.array_size.get()
# shape = (array_size.height, array_size.width)

# # Now, get the image and make it a 2-D array.
# im = adsimdet.image.array_data.get().reshape(shape)

# # Tell MatPlotLib to show the image.
# plt.imshow(im)

It's useful to make these steps into a Python function.

In [ ]:
# def show_ad_image(det):
#     array_size = det.image.array_size.get()
#     shape = (array_size.height, array_size.width)
#     im = det.image.array_data.get().reshape(shape)
#     plt.imshow(im)

In [ ]:
# plt.close()   # stop interaction with previous MPL window
# show_ad_image(adsimdet)

----
## Use HDF file saving plugin

Save image(s) to HDF5 file.

In [ ]:
from ophyd.areadetector import ADComponent
from ophyd.areadetector import EpicsSignalWithRBV
from ophyd.areadetector import HDF5Plugin
from ophyd.areadetector.filestore_mixins import FileStoreHDF5IterativeWrite

class MyHDF5Plugin(HDF5Plugin, FileStoreHDF5IterativeWrite):
    create_directory_depth = Component(EpicsSignalWithRBV, suffix="CreateDirectory")
    array_callbacks = Component(EpicsSignalWithRBV, suffix="ArrayCallbacks")

    pool_max_buffers = None
    
    def get_frames_per_point(self):
        return self.num_capture.get()

    def stage(self):
        super().stage()
        res_kwargs = {'frame_per_point': self.get_frames_per_point()}
        # res_kwargs = {'frame_per_point': self.num_capture.get()}
        self._generate_resource(res_kwargs)

In [ ]:
DATABROKER_ROOT_PATH = "/tmp/"

# note: AD path MUST, must, MUST have trailing "/"!!!
#  ...and... start with the same path defined in root (above)

# path as seen by detector IOC
WRITE_HDF5_FILE_PATH = "/tmp/simdet/%Y/%m/%d/"
#!!! NOTE !!! This filesystem is on the IOC (might be in a docker container)!

# path as seen by databroker
READ_HDF5_FILE_PATH = "/tmp/docker_ioc/iocadsky/tmp/simdet/%Y/%m/%d/"

In [ ]:
class MySingleTriggerSimDetector(SingleTrigger, SimDetector): 
       
    image = Component(ImagePlugin, suffix="image1:")
    hdf1 = ADComponent(
        MyHDF5Plugin,
        suffix='HDF1:', 
        root=DATABROKER_ROOT_PATH,
        write_path_template = WRITE_HDF5_FILE_PATH,
        read_path_template = READ_HDF5_FILE_PATH,
    )

In [ ]:
adsimdet = MySingleTriggerSimDetector(_ad_prefix, name='adsimdet')
adsimdet.stage_sigs["cam.num_images"] = 1
adsimdet.stage_sigs["cam.acquire_time"] = 0.1
adsimdet.stage_sigs["cam.acquire_period"] = 0.25
adsimdet.hdf1.stage_sigs["num_capture"] = 1

adsimdet.read_attrs.append("hdf1")
if adsimdet.hdf1.create_directory_depth.get() == 0:
    # probably not set, so let's set it now to some default
    adsimdet.hdf1.create_directory_depth.put(-5)

NOTE: EPICS AreaDetector file-saving plugins (such as 
the HDF plugin) must be primed before they can be 
used.  Priming must be done in these situations:

* the IOC has just started
* the image size has changed

If you do not prime the detector and its plugin chain, *ophyd* will report an *UnprimedPlugin* error like this if you try to trigger (acquire data from) the detector:

```
UnprimedPlugin: The plugin hdf1 on the area detector with name DETECTOR_NAME has not been primed.
```

To prime the HDF plugin, call its `warmup()` method.

In [ ]:
enabled = adsimdet.hdf1.enable.get()
adsimdet.hdf1.warmup()
adsimdet.hdf1.enable.put(enabled)

In [ ]:
adsimdet.hdf1.stage_sigs

In [ ]:
print(f"frames/point: {adsimdet.hdf1.get_frames_per_point()}")
RE(bp.count([adsimdet]))

----

Now, use databroker to retrieve that data and show the image.

In [ ]:
h = db[-1]

In [ ]:
for entry in h.documents():
    key, doc = entry
    print(key, doc)
    print("-"*40)

In [ ]:
#plt.close()   # stop interaction with previous MPL window
plt.imshow(h.xarray().adsimdet_image[0][0])

The structure of the HDF5 file as reported by the 
punx program (https://punx.readthedocs.io):

```
  entry:NXentry
    data:NXdata
      data:NX_UINT8[1,1024,1024] = [ ... ]
    instrument:NXinstrument
      NDAttributes:NXcollection
        NDArrayEpicsTSSec:NX_UINT32 = 958492486
        NDArrayEpicsTSnSec:NX_UINT32 = 334947107
        NDArrayTimeStamp:NX_FLOAT64 = 958492486.2348573
        NDArrayUniqueId:NX_INT32 = 3
      detector:NXdetector
        data:NX_UINT8[1,1024,1024] = [ ... ]
        NDAttributes:NXcollection
          ColorMode:NX_INT32 = 0
      performance
        timestamp:NX_FLOAT64[1,5] = [ ... ]
```

With attributes shown:

```
  entry:NXentry
    @NX_class = NXentry
    data:NXdata
      @NX_class = NXdata
      data:NX_UINT8[1,1024,1024] = [ ... ]
        @NDArrayDimBinning = 1
        @NDArrayDimOffset = 0
        @NDArrayDimReverse = 0
        @NDArrayNumDims = 2
        @signal = 1
    instrument:NXinstrument
      @NX_class = NXinstrument
      NDAttributes:NXcollection
        @NX_class = NXcollection
        @hostname = poof
        NDArrayEpicsTSSec:NX_UINT32 = 958492486
          @NDAttrDescription = The NDArray EPICS timestamp seconds past epoch
          @NDAttrName = NDArrayEpicsTSSec
          @NDAttrSource = Driver
          @NDAttrSourceType = NDAttrSourceDriver
        NDArrayEpicsTSnSec:NX_UINT32 = 334947107
          @NDAttrDescription = The NDArray EPICS timestamp nanoseconds
          @NDAttrName = NDArrayEpicsTSnSec
          @NDAttrSource = Driver
          @NDAttrSourceType = NDAttrSourceDriver
        NDArrayTimeStamp:NX_FLOAT64 = 958492486.2348573
          @NDAttrDescription = The timestamp of the NDArray as float64
          @NDAttrName = NDArrayTimeStamp
          @NDAttrSource = Driver
          @NDAttrSourceType = NDAttrSourceDriver
        NDArrayUniqueId:NX_INT32 = 3
          @NDAttrDescription = The unique ID of the NDArray
          @NDAttrName = NDArrayUniqueId
          @NDAttrSource = Driver
          @NDAttrSourceType = NDAttrSourceDriver
      detector:NXdetector
        @NX_class = NXdetector
        data:NX_UINT8[1,1024,1024] = [ ... ]
          @NDArrayDimBinning = 1
          @NDArrayDimOffset = 0
          @NDArrayDimReverse = 0
          @NDArrayNumDims = 2
          @signal = 1
        NDAttributes:NXcollection
          @NX_class = NXcollection
          ColorMode:NX_INT32 = 0
            @NDAttrDescription = Color mode
            @NDAttrName = ColorMode
            @NDAttrSource = Driver
            @NDAttrSourceType = NDAttrSourceDriver
      performance
        timestamp:NX_FLOAT64[1,5] = [ ... ]
```
